In [104]:
import pandas as pd
import numpy as np
import math

# Panda Config
pd.set_option('display.max_rows', 1000)

# Import and fix data
data = pd.read_csv('./raw-data/FantasyPros_2020_Overall_ADP_Rankings.csv')
columns = ['Rank', 'Player', 'Team', 'Bye', 'POS']
df = pd.DataFrame(data, columns=columns)
# df = df.set_index('Rank')
df.fillna(0)

# Draft Info
num_of_draft_spots = 12
draft_order = pd.Series(['Jack', 'Trevor', 'Luke', 'Shawnee', 'Fletcher', 'Jono', 'Henry', 'Kevin', 'Zach', 'Nate', 'Dillon', 'Wade'])

In [105]:
# Add Player Tiers Data

player_tiers_data = pd.read_csv('./data/player-draft-tiers.csv')
player_tiers_df = pd.DataFrame(player_tiers_data)

df = df.merge(player_tiers_df, on='Player', how='left')

In [106]:
# Add Estimated Draft Round and Pick number

def add_estimated_round_pick(row):
    if (row['Est. Round'] % 2 == 0):
        row['Est. Pick'] = num_of_draft_spots - ((row['Rank'] - 1) % num_of_draft_spots)
    else:
        row['Est. Pick'] = row['Rank'] % num_of_draft_spots
        if row['Est. Pick'] == 0:
            row['Est. Pick'] = 12
    return row

# Add 'Est. Round' column
df['Est. Round'] = df['Rank'].truediv(num_of_draft_spots).map(math.ceil, na_action='ignore')

# Add 'Est. Pick' column
df = df.apply(add_estimated_round_pick, axis=1)

# Add 'Name' column
df['Name'] = df['Est. Pick'].sub(1).map(draft_order)


In [107]:
trade_with = 9
give_rounds = [1, 2, 7]
receive_rounds = [1, 3, 4]

def add_trade_names(row):
    row['Trade Name'] = ''
    if (row['Est. Pick'] == trade_with and row['Est. Round'] in receive_rounds):
        row['Trade Name'] = 'Dillon'
    elif (row['Est. Pick'] == 11 and row['Est. Round'] in give_rounds):
        row['Trade Name'] = draft_order.get(trade_with - 1)
    
    return row

df = df.apply(add_trade_names, axis=1)

In [108]:
# Add Player Notes Data

player_notes_data = pd.read_csv('./data/player-notes.csv')
player_notes_df = pd.DataFrame(player_notes_data)

player_notes_df

,Player,NOTES
0,Christian McCaffrey,You can try to make the argument that McCaffre...
1,Saquon Barkley,"After an extremely impressive rookie season, B..."
2,Ezekiel Elliott,There‘s only one running back over the last fi...
3,Derrick Henry,"Yes, Derrick Henry finished with 1,540 yards a..."
4,Dalvin Cook,It turns out Gary Kubiak is very good for a ru...
5,Alvin Kamara,There‘s just one running back who‘s had a high...
6,Nick Chubb,"Through 10 games last year, Chubb was doing ve..."
7,Joe Mixon,Mixon‘s value was up in the air for a week or ...
8,Josh Jacobs,We‘ve continually heard that Jacobs is going t...
9,Clyde Edwards-Helaire,I had Edwards-Helaire as my top running back i...


In [110]:
def shade_row(row):
    def bg(color):
        return f"background-color: {color}"
    
    def color(clr):
        return f"color: {clr}"
    
    def color_pos(pos):
        if isinstance(pos, str):
            if 'RB' in pos:
                return 'rgb(240, 230, 250)'
            if 'QB' in pos:
                return 'rgb(250, 240, 230)'
            if 'WR' in pos:
                return 'rgb(250, 220, 220)'
            if 'TE' in pos:
                return 'rgb(240, 250, 240)'
        return 'transparent'
            
    def color_name(name):
        if name == 'Dillon':
            return 'red'
        return 'black'
    
    def color_tier(tier):
        if tier >= 7:
            return 'black'
        if tier >= 6:
            return 'purple'
        if tier >= 5:
            return 'blue'
        if tier >= 4:
            return 'green'
        if tier >= 3:
            return 'yellowgreen'
        if tier >= 2:
            return 'goldenrod'
        if tier > 1:
            return 'orange'
        if tier == 1:
            return 'red'
        return 'black'
    
    return ['', '', '', '', bg(color_pos(row['POS'])), color(color_tier(row['Tier'])), '', '', '', color(color_name(row['Name'])), color(color_name(row['Trade Name']))]
    

df.style.apply(shade_row, axis=1)

,Rank,Player,Team,Bye,POS,Tier,Round,Est. Round,Est. Pick,Name,Trade Name
0,1,Christian McCaffrey,CAR,13.000000,RB1,1.000000,Round 1,1,1,Jack,
1,2,Ezekiel Elliott,DAL,10.000000,RB2,1.000000,Round 1,1,2,Trevor,
2,3,Saquon Barkley,NYG,11.000000,RB3,1.000000,Round 1,1,3,Luke,
3,4,Derrick Henry,TEN,7.000000,RB4,1.000000,Round 1,1,4,Shawnee,
4,5,Michael Thomas,NO,6.000000,WR1,1.000000,Top 20 Overall,1,5,Fletcher,
5,6,Dalvin Cook,MIN,7.000000,RB5,1.500000,Round 1,1,6,Jono,
6,7,Alvin Kamara,NO,6.000000,RB6,1.000000,Round 1,1,7,Henry,
7,8,Davante Adams,GB,5.000000,WR2,1.000000,Top 20 Overall,1,8,Kevin,
8,9,Nick Chubb,CLE,9.000000,RB7,1.500000,Round 1,1,9,Zach,Dillon
9,10,Tyreek Hill,KC,10.000000,WR3,1.000000,Top 20 Overall,1,10,Nate,
